# IEMS 308 Homework 3 - Melissa Melnick

# Importing and Data Cleaning

In [301]:
import os
import re
import string
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

Read in the 2013 Articles

In [8]:
bi2013 = []
folder = '../BI-articles/2013/'

for article in os.listdir(folder):
    with open(folder + '//' + article, 'r', encoding = 'latin-1') as art13:
        lines = art13.read()
        bi2013.append(lines)
    art13.close()


Read in the 2014 Articles

In [9]:
bi2014 = []
folder = '../BI-articles/2014/'

for article in os.listdir(folder):
    with open(folder + '//' + article, 'r', encoding = 'latin-1') as art14:
        lines = art14.read()
        bi2014.append(lines)
    art14.close()

In [10]:
#combine all of the articles
bi_all = bi2013 + bi2014

Reading in Training Data Sets

Segment the data into Sentences

In [12]:
sents = []
for article in bi_all:
    sent_token = sent_tokenize(article)
    for sentence in sent_token:
        sents.append(sentence)

In [262]:
sents[14]

'SHIBOR at 25% basically means there is no functioning interbank market in China - it\'s like money markets seizing up post-Lehman China faces "Das Boot" dilemma: Do you face the depth charges or dive deeper?'

Segment the data into individual word tokens AND remove stop words

In [13]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/melmelnick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/melmelnick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/melmelnick/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
words = []
stop_words = set(stopwords.words("english"))
for article in bi_all:
    tokens = word_tokenize(article)
    for word in tokens:
        words.append(word)

result = [i for i in tokens if not i in stop_words]

Stemming

In [15]:
stemmer = PorterStemmer()
result_stemming = [stemmer.stem(word) for word in result]


['port',
 'lo',
 'angeleslo',
 'angel',
 '(',
 'reuter',
 ')',
 '-',
 'crippl',
 'cargo',
 'backup',
 'u.s.',
 'west',
 'coast',
 'port',
 'stretch',
 'third',
 'month',
 'amid',
 'industri',
 'report',
 'thursday',
 'prolong',
 'shipment',
 'delay',
 'good',
 'rang',
 'yoga',
 'apparel',
 'rice',
 'nba',
 'bobblehead',
 'collect',
 'frozen',
 'french',
 'fri',
 '.',
 'the',
 'congest',
 'crisi',
 'pronounc',
 'twin',
 'port',
 'lo',
 'angel',
 'long',
 'beach',
 ',',
 'nation',
 "'s",
 'two',
 'busiest',
 'cargo',
 'hub',
 ',',
 'marin',
 'offici',
 'report',
 '11',
 'ship',
 'anchor',
 'thursday',
 'wait',
 'berth',
 'open',
 '.',
 'the',
 'number',
 'freighter',
 'kept',
 'wait',
 'outsid',
 'two',
 'port',
 'fluctuat',
 'eight',
 '18',
 'given',
 'day',
 'sinc',
 'slowdown',
 'began',
 'around',
 'mid-octob',
 ',',
 'said',
 'phillip',
 'sanfield',
 ',',
 'spokesman',
 'port',
 'lo',
 'angel',
 '.',
 'smaller',
 'backup',
 'hit',
 'west',
 'coast',
 'port',
 ',',
 'includ',
 'oakla

# Part 1: Percentage Extraction

Building Regex Statements that will extract all Percentages

In [189]:
#percentages training data set
pct = pd.read_csv('../labels/percentage.csv', delimiter=',', encoding='latin-1', header=None, names=["Percentages Training Data"])

In [599]:
#percentage made up of digits, a decimal point, and a percent sign (e.g. 9.5%)
r1 = r"[0-9]*[\.]?[0-9]+[\%]"

#percentage made up of digits and the word percent (e.g. 9.5 percent)
r2 = r"[0-9]*[\.]?[0-9]+ percent"

#percentage made up of digits and the words percentage points (e.g. 9.5 percentage points)
r3 = r"[0-9]*[\.]?[0-9]+ percentage point[s]?"

#covers percentages written in plain english
r4 = r'((?:one|two|twe|thr|thi|for|fou|fif|fiv|six|sev|eig|nin|ten|zer|hun|hal|thou|quart|)[a-z]+ percent)'


pre_list = [r1,r2,r3,r4]

Finding Matches within the Actual Business Insider Articles

In [587]:
p1 = []
p2 = []
p3 = []
p4 = []

match_lists = [p1, p2, p3, p4]

In [597]:
for sent in sents:
    for i in range(0,len(pre_list)):
        match = re.findall(pre_list[i], sent)
        if match != [] and match != ['']:
            match_lists[i].append(match)
        
for p in match_lists:
    p = [i for i in p if i != []]

In [602]:
all_percents = []
for plist in match_lists:
    for indiv_list in plist:
        for item in indiv_list:
            if item != "":
                all_percents.append(item)

In [606]:
pd.Series(list(all_percents)).to_csv("found_percents.csv", header = False, index = False)

# Part 2 CEO Extraction

Cleaning CEO data

In [257]:
#CEOs
ceos_orig = pd.read_csv('../labels/ceo.csv', delimiter=',', encoding='latin-1', header=None).values.tolist()
ceos = []

#turning the ceos into a list of strings with each unique CEO name
for name_list in ceos_orig:
    name_str = ''
    for i in range(len(name_list)):
        name_str += str(name_list[i]) + ' '
    name_str = name_str.rstrip()
    if name_str not in ceos:
        ceos.append(name_str)



In [432]:
#companies
companies_orig = pd.read_csv('../labels/companies.csv', delimiter = ',', encoding = 'latin-1', header = None).values.tolist()
companies = []

for comp_list in companies_orig:
    comp_str = ''
    for i in range(len(comp_list)):
        comp_str += str(comp_list[i]) + ' '
    comp_str = comp_str.rstrip()
    if comp_str not in companies:
        companies.append(comp_str)


Building Vectors with features that will Signal a CEO

Features to explore:

Number of characters in sentence 

Number or words in sentence

Number of words in Potential CEO

Number of characters of potential CEO

Number of Capital Letters in potential CEO

Number of Capital Letters in sentence

Number of punctuation characters in potental CEO

If the sentence contains a word pertaining to CEO

If the sentence contains a word pertaining to a company

In [385]:
not_ceo_words = ['CFO', 'COO', 'cfo', 'coo', 'CTO', 'cto', 'VP', 'vp', 'vice', 'manager',
                    'Inc', 'inc', 'Corp', 'org', 'Organization', 'corp']

ceo_words = ['Chief', 'chief', 'Executive', 'executive',"Officer", 'officer', 'CEO', 'ceo']

def build_vector(word_sent_group):
    
    word = word_sent_group[0]
    sentence = word_sent_group[1]
    
    #number of characters in sentence
    num_char_sent = len(sentence)
    
    #number of words in sentence
    num_word_sent = len(sentence.split(' '))
    
    #number of characters in potential CEO
    num_char_ceo = len(word)
    
    #number of words in ceo:
    num_word_ceo = len(word.split(' '))
    
    #number of capital letters in ceo:
    ceo_caps = 0
    for char in word:
        if char.isupper() == True:
            ceo_caps += 1
    
    #number of capital letters in sentence
    sent_caps = 0
    for char in sentence:
        if char.isupper() == True:
            sent_caps += 1
            
    #number of punctuation characters in CEO
    ceo_punct = 0
    for char in word:
        if char in string.punctuation:
            ceo_punct += 1
            
    #number of punctuation characters in sentence
    sent_punct = 0
    for char in sentence:
        if char in string.punctuation:
            sent_punct += 1
            
    # 1 if sentence contains a word pertaining to a CEO
    ceo_word = 0
    for word in sentence.split(' '):
        if word in ceo_words:
            ceo_word = 1
            break
        else:
            continue
            
    # 1 if sentence contains a word pertaining to a company (aka NOT a CEO)
    not_ceo_word = 0
    for word in sentence.split(' '):
        if word in not_ceo_words:
            ceo_word = 1
            break
        else:
            continue
    

    feature_vector = [num_char_sent, num_word_sent, num_char_ceo, num_word_ceo, 
                      sent_caps, ceo_caps, ceo_punct, sent_punct, ceo_word, not_ceo_word]
    return feature_vector

In [360]:
#remove stop words from sents
clean_sents = []
stop_words = set(stopwords.words("english"))
for sentence in sents:
    temp = sentence.split(' ')
    clean_sent= ""
    for word in temp:
        if word not in stop_words:
            clean_sent += word + ' '
    clean_sents.append(clean_sent.rstrip())


"ReutersChina's seven day repo rose record high 10.77% Shanghai, highest since March 2003, according Bloomberg*."

In [399]:
#find matches of things that could either be companies or people
maybe_people = []
for i in range(len(clean_sents)):
    caps = re.findall(r"[A-Z][a-z]* [A-Z][a-z]*", clean_sents[i])
    if caps !=[]:
        for match in caps:
            maybe_people.append([match, clean_sents[i]])

In [400]:
maybe_people

[['And Zerohedge', 'And Zerohedge reported overnight repo hit 25%.'],
 ['Dragon Boat',
  'The liquidity squeeze China first began ahead Dragon Boat festival earlier month.'],
 ['But Diana',
  'Spikes interbank rates common right holidays.Â\xa0 But Diana Choyleva Lombard Street Research said symptomatic bigger problem.'],
 ['Choyleva Lombard',
  'Spikes interbank rates common right holidays.Â\xa0 But Diana Choyleva Lombard Street Research said symptomatic bigger problem.'],
 ['Street Research',
  'Spikes interbank rates common right holidays.Â\xa0 But Diana Choyleva Lombard Street Research said symptomatic bigger problem.'],
 ['Bank America',
  'In new note clients Bank America\'s Ting Lu wrote: "There many factors behind interbank liquidity squeeze might cited, believe ultimate reason central bankâ\x80\x99s tough stance PBOC practically provide unlimited liquidity ease every squeeze wishes to.'],
 ['Ting Lu',
  'In new note clients Bank America\'s Ting Lu wrote: "There many factors beh

In [502]:
more_not_ceo_words = ['Rio', 'Sao', 'Paolo', 'China', 'Commission', 'Vice', 'President',
                    'Operations', 'National', 'Finance', 'Chairman', 'Bloomberg', 'Street', 
                     'Social', 'Exchange', 'Federal', 'Reserve', 'Company', 'Saudi', 'Arabia',
                    'Switzerland', 'New', 'York', 'Los', 'Angeles', 'San', 'Fansisco', 'India',
                    'Brazil', 'Russia', 'France', 'Goldman', 'Director', 'Japan','Prime', 
                    'Minister', 'European', 'Europe', 'Politics', 'Editor', 'Senator', 'Board', 
                     'Managemnt' 'Commission', 'Advisory', 'East', 'North', 'South', 'West', 
                    'Senator', 'Social', 'Security','Africa' 'American', 'Jersey', 'United', 
                    'States', 'Kingdom', 'Governmemt', 'Capital', 'Turkey', 'Department', 
                    'Business', 'Parliament', 'Fed', 'The', 'Committee', 'Economic', 'Congress',
                    'Japanese', 'Secretary', 'Treasury', "And", 'Bank', 'Labor', 'Quantitative', 
                      'Diversity', 'Fixed', 'Income', 'Currency', 'From', 'Ex', 'July', 'April',
                     'Paris', 'World', 'American', 'Act', 'Task', 'Force', 'Live', "House"]


In [423]:
#filter out non ceo words from the maybe people matches
maybe_people_filtered = []
for i in range(len(maybe_people)):
    temp = maybe_people[i][0].split(' ')
    if (temp[0] not in not_ceo_words) and (temp[0] not in more_not_ceo_words) and (temp[1] not in not_ceo_words) and (temp[1] not in more_not_ceo_words):
            maybe_people_filtered.append(maybe_people[i])

In [424]:
maybe_people_filtered

[['Dragon Boat',
  'The liquidity squeeze China first began ahead Dragon Boat festival earlier month.'],
 ['But Diana',
  'Spikes interbank rates common right holidays.Â\xa0 But Diana Choyleva Lombard Street Research said symptomatic bigger problem.'],
 ['Choyleva Lombard',
  'Spikes interbank rates common right holidays.Â\xa0 But Diana Choyleva Lombard Street Research said symptomatic bigger problem.'],
 ['Ting Lu',
  'In new note clients Bank America\'s Ting Lu wrote: "There many factors behind interbank liquidity squeeze might cited, believe ultimate reason central bankâ\x80\x99s tough stance PBOC practically provide unlimited liquidity ease every squeeze wishes to.'],
 ['So Chinese', 'So Chinese central bank stepping in?'],
 ['Charlene Chu',
  "Earlier week Fitch's Charlene Chu warned thatÂ\xa0 China's credit bubble unlike anything modern history."],
 ['Das Boot',
  'SHIBOR 25% basically means functioning interbank market China - like money markets seizing post-Lehman China faces "

In [426]:
vectors = []
for grouping in maybe_people_filtered:
    vectors.append(build_vector(grouping)) 

In [429]:
vectors[100:105]

[[248, 39, 8, 2, 38, 2, 0, 22, 0, 0],
 [248, 39, 7, 2, 38, 2, 0, 22, 0, 0],
 [325, 40, 13, 2, 7, 2, 0, 8, 0, 0],
 [325, 40, 8, 2, 7, 2, 0, 8, 0, 0],
 [149, 17, 16, 2, 10, 2, 0, 4, 0, 0]]

In [434]:
#compare with the training set to form positive and negative matches
neg_matches = []
pos_matches = []

#negative matches
for company in companies:
    for sentence in clean_sents:
        if company in sentence:
            neg_matches.append([company, sentence])

In [438]:
#positive matches
for ceo in ceos:
    for sentence in clean_sents:
        if ceo in sentence:
            pos_matches.append([ceo, sentence])

In [462]:
#building the vectors

#negative vectors
neg_vectors = []
for grouping in neg_matches:
    neg_vectors.append(build_vector(grouping))
    
for vector in neg_vectors:
    vector.append(0)

In [463]:
#positive vectors
pos_vectors = []
for grouping in pos_matches:
    pos_vectors.append(build_vector(grouping))

for vector in pos_vectors:
    vector.append(1)

In [443]:
#potential ceos
maybe_ceo_vectors = []
for grouping in maybe_people_filtered:
    maybe_ceo_vectors.append(build_vector(grouping))

In [478]:
#combining data
all_data = pd.DataFrame(pos_vectors + neg_vectors)
x_values = all_data.drop(10, axis = 1)
y_values = all_data[10]

In [501]:
from sklearn import tree
#training decision tree
ceo_dt = tree.DecisionTreeClassifier()
ceo_dt = ceo_dt.fit(x_values, y_values)

In [495]:
test_data = pd.DataFrame(maybe_ceo_vectors)

In [574]:
predict_ceos = ceo_dt.predict(test_data)

In [575]:
predict_ceos

array([1, 0, 0, ..., 1, 0, 0])

Using a Decision Tree as the Model for this Problem


In [578]:
yes_ceos = []
for index, classification in enumerate(predict_ceos):
    if classification == 1:
        yes_ceos.append(maybe_people_filtered[index][0])

In [585]:
found_ceos = set(yes_companies)
output = set(list(found_ceos) + list(ceos))
pd.Series(list(output)).to_csv("found_ceos.csv", header = False, index = False)

# Part 3: Company Extraction

We cleaned the training data for Ceos and Companies in the previous part so we can go right into making the feature vectors

In [521]:
#list of not company words
not_comp_words = ['Chief', 'Executive', 'Officer', 'Director', 'Chairman', 'President', 'Vice', 
               'Secretary', 'Treasurer','Rio', 'Sao', 'Paolo', 'China', 'Commission', 'Vice', 'President',
                    'Operations', 'National', 'Finance', 'Chairman', 'Bloomberg', 'Street', 
                     'Social', 'Exchange', 'Federal', 'Reserve', 'Company', 'Saudi', 'Arabia',
                    'Switzerland', 'New', 'York', 'Los', 'Angeles', 'San', 'Fansisco', 'India',
                    'Brazil', 'Russia', 'France', 'Director', 'Japan','Prime', 
                    'Minister', 'European', 'Europe', 'Politics', 'Editor', 'Senator', 'Board', 
                     'Managemnt' 'Commission', 'Advisory', 'East', 'North', 'South', 'West', 
                    'Senator', 'Social', 'Security','Africa' 'American', 'Jersey', 'United', 
                    'States', 'Kingdom', 'Governmemt', 'Capital', 'Turkey', 'Department', 
                    'Business', 'Parliament', 'Fed', 'The', 'Committee', 'Economic', 'Congress',
                    'Japanese', 'Secretary', 'Treasury', "And", 'Bank', 'Labor', 'Quantitative', 
                      'Diversity', 'Fixed', 'Income', 'Currency', 'From', 'Ex', 'July', 'April',
                     'Paris', 'World', 'American', 'Act', 'Task', 'Force', 'Live', "House", 'The',
                 ]
    
comp_words = ['Inc', 'Corp', 'Org', 'Organization', 'Ltd']

In [ ]:
#function for company feature vectors
def build_vector_company(word_sent_group):
    
    word = word_sent_group[0]
    sentence = word_sent_group[1]
    
    #number of characters in sentence
    num_char_sent = len(sentence)
    
    #number of words in sentence
    num_word_sent = len(sentence.split(' '))
    
    #number of characters in potential CEO
    num_char_comp = len(word)
    
    #number of words in ceo:
    num_word_comp = len(word.split(' '))
    
    #number of capital letters in ceo:
    comp_caps = 0
    for char in word:
        if char.isupper() == True:
            comp_caps += 1
    
    #number of capital letters in sentence
    sent_caps = 0
    for char in sentence:
        if char.isupper() == True:
            sent_caps += 1
            
    #number of punctuation characters in CEO
    comp_punct = 0
    for char in word:
        if char in string.punctuation:
            comp_punct += 1
            
    #number of punctuation characters in sentence
    sent_punct = 0
    for char in sentence:
        if char in string.punctuation:
            sent_punct += 1
            
    # 1 if sentence contains a word pertaining to a CEO
    comp_word = 0
    for word in sentence.split(' '):
        if word in comp_words:
            comp_word = 1
            break
        else:
            continue
            
    # 1 if sentence contains a word pertaining to a company (aka NOT a CEO)
    not_comp_word = 0
    for word in sentence.split(' '):
        if word in not_comp_words:
            comp_word = 1
            break
        else:
            continue
    

    feature_vector = [num_char_sent, num_word_sent, num_char_comp, num_word_comp, 
                      sent_caps, comp_caps, comp_punct, sent_punct, comp_word, not_comp_word]
    return feature_vector

In [522]:
#find matches of things that could either be companies or people
maybe_comps = []
for i in range(len(clean_sents)):
    caps = re.findall(r"([A-Z][a-z]+(?=\s[A-Z])(?:\s[A-Z][a-z]+)+)", clean_sents[i])
    if caps !=[]:
        for match in caps:
            maybe_comps.append([match, clean_sents[i]])

In [520]:
maybe_comps

[['And Zerohedge', 'And Zerohedge reported overnight repo hit 25%.'],
 ['Dragon Boat',
  'The liquidity squeeze China first began ahead Dragon Boat festival earlier month.'],
 ['But Diana Choyleva Lombard Street Research',
  'Spikes interbank rates common right holidays.Â\xa0 But Diana Choyleva Lombard Street Research said symptomatic bigger problem.'],
 ['Bank America',
  'In new note clients Bank America\'s Ting Lu wrote: "There many factors behind interbank liquidity squeeze might cited, believe ultimate reason central bankâ\x80\x99s tough stance PBOC practically provide unlimited liquidity ease every squeeze wishes to.'],
 ['Ting Lu',
  'In new note clients Bank America\'s Ting Lu wrote: "There many factors behind interbank liquidity squeeze might cited, believe ultimate reason central bankâ\x80\x99s tough stance PBOC practically provide unlimited liquidity ease every squeeze wishes to.'],
 ['So Chinese', 'So Chinese central bank stepping in?'],
 ['Bank China',
  "As previously exp

In [523]:
#filter out non ceo words from the maybe people matches
maybe_comps_filtered = []
for i in range(len(maybe_comps)):
    temp = maybe_people[i][0].split(' ')
    check = any(item in temp for item in not_comp_words)
    if check == False:
        maybe_comps_filtered.append(maybe_comps[i])
    
        

In [528]:
#build positive and negative examples
#compare with the training set to form positive and negative matches
neg_comp_matches = []
pos_comp_matches = []

#positive matches
for company in companies:
    for sentence in clean_sents:
        if company in sentence:
            pos_comp_matches.append([company, sentence])

In [529]:
#negative matches
for ceo in ceos:
    for sentence in clean_sents:
        if ceo in sentence:
            neg_comp_matches.append([ceo, sentence])

In [532]:
#building the vectors

#negative vectors
neg_comp_vectors = []
for grouping in neg_comp_matches:
    neg_comp_vectors.append(build_vector(grouping))
    
for vector in neg_comp_vectors:
    vector.append(0)

In [533]:
#positive vectors
pos_comp_vectors = []
for grouping in pos_matches:
    pos_comp_vectors.append(build_vector(grouping))

for vector in pos_comp_vectors:
    vector.append(1)

In [557]:
#potential companies
maybe_comp_vectors = []
for grouping in maybe_comps_filtered:
    maybe_comp_vectors.append(build_vector(grouping))

In [558]:
#combining data
all_comp_data = pd.DataFrame(pos_vectors + neg_vectors)
x_comp_values = all_data.drop(10, axis = 1)
y_comp_values = all_data[10]

In [535]:
all_comp_data

,0,1,2,3,4,5,6,7,8,9,10
0,77,11,10,2,6,2,0,6,0,0,1
1,104,16,10,2,11,2,0,3,1,0,1
2,62,9,10,2,6,2,0,3,0,0,1
3,47,9,10,2,6,2,0,1,0,0,1
4,255,34,10,2,7,2,0,8,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
205274,157,22,3,1,16,3,0,6,0,0,0
205275,64,10,3,1,8,3,0,3,0,0,0
205276,68,11,3,1,6,3,0,1,0,0,0
205277,49,8,3,1,7,3,0,2,0,0,0


In [580]:
comp_dt = tree.DecisionTreeClassifier()
comp_dt = comp_dt.fit(x_comp_values, y_comp_values)

In [581]:
comp_test_data = pd.DataFrame(maybe_comp_vectors)
predict_comps = comp_dt.predict(comp_test_data)

In [582]:
yes_companies = []
for index, classification in enumerate(predict_comps):
    if classification == 1:
        yes_companies.append(maybe_comps[index][0])

In [584]:
yes_companies

['And Zerohedge',
 'Bank America',
 'So Chinese',
 'Charlene Chu',
 'Lehman China',
 'Michael Mc',
 'Senior Vice President People Operations Laszlo Bock',
 'Viz This',
 'China Flash',
 'Hongbin Qu',
 'Markit Economics Here',
 'The National Bureau Statistics',
 'The Nikkei',
 'Maury Harris',
 'Mohamed El',
 'Business Insider',
 'Growth In Assets Last Year',
 'Goldman Sachs Oppenheimer Funds',
 'Without Fail',
 'Gluskin Sheff',
 'David Rosenberg',
 'Gluskin Sheff',
 'Jeff Gundlach',
 'Santelli Exchange',
 'Federal Reserve',
 'Federal Reserve Chairman Ben Bernanke',
 'Federal Reserve Washington',
 'Volatility Japanese',
 'Frank Polich',
 'Now Dow',
 'Fired Men',
 'The Board',
 'The Company',
 'Annual Meeting',
 'Annual Meeting',
 'Frank Polich',
 'The Fed',
 'Federal Open Market Committee',
 'The Committee',
 'The Committee',
 'The Committee',
 'When Committee',
 'James Bullard',
 'The Federal Reserve',
 'Federal Reserve\nBelow',
 'Federal Reserve',
 'Federal Reserve',
 'Federal Reserve\n

In [564]:
found_companies = set(yes_companies)
output = set(list(found_companies) + list(companies))
pd.Series(list(output)).to_csv("found_companies.csv", header = False, index = False)